In [6]:
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
import seaborn as sns 
from datetime import datetime
from scipy import stats
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import make_scorer, mean_squared_error
from sklearn.linear_model import Ridge
from xgboost import XGBRFRegressor
from itertools import combinations
from xgboost import XGBClassifier
import re
import warnings
warnings.filterwarnings("ignore")

In [7]:
Marital_status = 1
Appl_mode = 1
Appl_order = 1
course = 9238
Dayt_even_att = 1 
Nation = 1
Mother_ocup = 5 
Father_ocup = 5
Displaced = 0
Edu_esp = 0
Deptor = 0
Tuiton_feup_td = 1 
Gender = 0
Scolarsip_hol = 1 
International = 0
Pre_qualifi_grade = 126.0 
Pre_qualification = 1
Adgrade = 122.6
Age_enrll = 18
Unemply_rate = 11.1 
Infl_rate = 0.6
GDP = 2.02
Mother_qualif = 1
Father_qualif = 19
Cur_units1_cred = 0
Cur_units1_enrll = 6
Cur_units1_evalt = 6
Cur_units1_aprob = 6
Cur_units1_grade = 14.5
Cur_units1_weval = 0
Cur_units2_cred = 0
Cur_units2_enrll = 6
Cur_units2_evalt  = 7
Cur_units2_aprob = 6
Cur_units2_grade = 12.42
Cur_units2_weval = 0

In [9]:
numericas = pd.DataFrame({'Previous qualification (grade)':[Pre_qualifi_grade],"Previous qualification":[Pre_qualification],'Admission grade':[Adgrade],'Age at enrollment':[Age_enrll],
"Unemployment rate":[Unemply_rate],"Inflation rate":[Infl_rate],"GDP":[GDP],"Mother's qualification":[Mother_qualif],
"Father's qualification":[Father_qualif], 'Admission grade':[Adgrade],  'Curricular units 1st sem (credited)':[Cur_units1_cred],
'Curricular units 1st sem (enrolled)':[Cur_units1_enrll],'Curricular units 1st sem (evaluations)':[Cur_units1_evalt],'Curricular units 1st sem (approved)':[Cur_units1_aprob],
'Curricular units 1st sem (grade)':[Cur_units1_grade],'Curricular units 1st sem (without evaluations)':[Cur_units1_weval],'Curricular units 2st sem (credited)':[Cur_units2_cred],
'Curricular units 2st sem (enrolled)':[Cur_units2_enrll],'Curricular units 2st sem (evaluations)':[Cur_units2_evalt],'Curricular units 2st sem (approved)':[Cur_units2_aprob],
'Curricular units 2st sem (grade)':[Cur_units2_grade], 'Curricular units 2st sem (without evaluations)':[Cur_units2_weval]})
numericas

,Previous qualification (grade),Previous qualification,Admission grade,Age at enrollment,Unemployment rate,Inflation rate,GDP,Mother's qualification,Father's qualification,Curricular units 1st sem (credited),...,Curricular units 1st sem (evaluations),Curricular units 1st sem (approved),Curricular units 1st sem (grade),Curricular units 1st sem (without evaluations),Curricular units 2st sem (credited),Curricular units 2st sem (enrolled),Curricular units 2st sem (evaluations),Curricular units 2st sem (approved),Curricular units 2st sem (grade),Curricular units 2st sem (without evaluations)
0,126.0,1,122.6,18,11.1,0.6,2.02,1,19,0,...,6,6,14.5,0,0,6,7,6,12.42,0


In [10]:
# Streamlit app
st.title("Predicción Estado de estudiante de la APP basada en características de usuario")

# Entradas del usuario
Marital_status = st.selectbox("Seleccione su estado civil:", [1,2,3,4,5,6])
Appl_mode = st.selectbox("Seleccione el modo de aplicación:", [1,2,3,4,5,6,7,9,10,12,15,16,17,18,26,27,35,39,42,43,44,51,53])
Appl_order = st.selectbox("Seleccione orden de aplicación:", [0,1,2,3,4,5,6,9])
course = st.selectbox("Seleccione orden de aplicación:", [33,39,171,979,8014,9003,9070,9085,9119,9130,9147,9238,9254,9500,9556,9670,9773,9853,9991])
Dayt_even_att = st.selectbox("Seleccione sistencia diurna o nocturna :", [0,1])
Nation = st.selectbox("Seleccione orden de aplicación:", [1,2,6,11,17,21,22,24,25,26,32,41,62,100,101,103,105,109])
Mother_ocup = st.selectbox("Seleccione ocupación madre:", [0,1,2,3,4,5,6,7,8,9,10,11,38,90,99,101,103,122,123,124,125,127,131,132,134,141,143,144,151,152,153,163,171,172,173,175,191,192,193,194])
Father_ocup = st.selectbox("Seleccione ocupación padre:", [0,1,2,3,4,5,6,7,8,9,10,11,12,13,19,22,39,90,96,99,101,102,103,112,114,121,122,123,124,125,131,132,134,135,141,143,144,148,151,152,153,154,161,163,171,172,174,175,181,182,183,191,193,193,194,195])
Displaced = st.selectbox("Seleccione si es desplazado :", [0,1])
Edu_esp = st.selectbox("Seleccione si necesita educación especial:", [0,1])
Deptor = st.selectbox("Seleccione si es deudor:", [0,1])
Tuiton_feup_td = st.selectbox("Seleccione si tiene matricula al día :", [0,1])
Gender = st.selectbox("Seleccione Género :", [0,1])
Scolarsip_hol = st.selectbox("Seleccione si tiene beca:", [0,1])
International = st.selectbox("Seleccione si es extranjero:", [0,1])

# Campos numéricos sin valor predeterminado
Pre_qualifi_grade = st.text_input("Ingrese la calificación previa (grado):", value="")
Pre_qualification = st.text_input("Ingrese la calificación previa:", value="")
Adgrade = st.text_input("Ingrese grado de admision :", value="")
Age_enrll = st.text_input("Ingrese Edad de inscripción:", value="")
Unemply_rate = st.text_input("Ingrese Tasa de desempleo :", value="")
Infl_rate = st.text_input("Ingrese Tasa de inflación :", value="")
GDP = st.text_input("Ingrese PIB :", value="")
Mother_qualif = st.text_input("Ingrese Calificación madre :", value="")
Father_qualif = st.text_input("Ingrese Calificación padre :", value="")
Cur_units1_cred = st.text_input("Ingrese número de unidades curriculares en el 1 semestre (acreditado) :", value="")
Cur_units1_enrll = st.text_input("Ingrese número de unidades curriculares en el 1 semestre (matriculado) :", value="")
Cur_units1_evalt = st.text_input("Ingrese número de unidades curriculares en el 1 semestre (con evaluaciones) :", value="")
Cur_units1_aprob = st.text_input("Ingrese número de unidades curriculares en el 1 semestre (aprobado) :", value="")
Cur_units1_grade = st.text_input("Ingrese número de unidades curriculares en el 1 semestre (grado) :", value="")
Cur_units1_weval = st.text_input("Ingrese número de unidades curriculares en el 1 semestre (sin evaluaciones) :", value="")
Cur_units2_cred = st.text_input("Ingrese número de unidades curriculares en el 2 semestre (acreditado) :", value="")
Cur_units2_enrll = st.text_input("Ingrese número de unidades curriculares en el 2 semestre (matriculado) :", value="")
Cur_units2_evalt = st.text_input("Ingrese número de unidades curriculares en el 2 semestre (con evaluaciones) :", value="")
Cur_units2_aprob = st.text_input("Ingrese número de unidades curriculares en el 2 semestre (aprobado) :", value="")
Cur_units2_grade = st.text_input("Ingrese número de unidades curriculares en el 2 semestre (grado) :", value="")
Cur_units2_weval = st.text_input("Ingrese número de unidades curriculares en el 2 semestre (sin evaluaciones) :", value="")


# Convertir los valores de texto a números si es posible
if st.button("Calcular"):
    try:
        Avg = float(Avg)
        Time_App = float(Time_App)
        Time_Web = float(Time_Web)
        Length = float(Length)

        # Generar datos de entrada
        numericas = generar_numericas(Pre_qualifi_grade, Pre_qualification, Adgrade, Age_enrll, Unemply_rate, Infl_rate, GDP, Mother_qualif, Father_qualif, Cur_units1_cred, Cur_units1_enrll, Cur_units1_evalt, Cur_units1_aprob, Cur_units1_grade, Cur_units1_weval,Cur_units2_cred, Cur_units2_enrll, Cur_units2_evalt, Cur_units2_aprob, Cur_units2_grade, Cur_units2_weval)
        numericas, dum = generar_dummies(Marital_status, Appl_mode, Appl_order, course, Dayt_even_att, Nation, Mother_ocup, Father_ocup, Displaced,Edu_esp, Deptor, Tuiton_feup_td, Gender, Scolarsip_hol,International, numericas)

        # Concatenar los dataframes
        covariables = ['Curricular units 2 sem (approved)_2', 'Curricular units 2 sem (evaluations)_2', 'Curricular units 2 sem (enrolled)_2',
                    'Curricular units 2 sem (approved)__Curricular units 2 sem (grade)', 'Admission grade_Curricular units 2 sem (approved)', 
                    'Curricular units 1 sem (approved)__Curricular units 2 sem (approved)', 'Curricular units 2 sem (approved)__coc__Curricular units 1 sem (enrolled)', 
                    'Curricular units 2 sem (approved)__coc__Curricular units 2 sem (enrolled)', 'Curricular units 2 sem (approved)__coc__Age at enrollment',
                    'Tuition fees up to date__Scholarship holder',
                    'Educational special needs__Tuition fees up to date', 'Gender__Scholarship holder', 'Curricular units 2 sem (approved)_Tuition fees up to date_1',
                    'Curricular units 2 sem (evaluations)_Tuition fees up to date_1',
                    'Curricular units 1 sem (approved)_Tuition fees up to date_1']

        base_modelo2 = pd.concat([numericas, dum], axis=1)
        base_modelo2 = base_modelo2[covariables]

        # Predicción
        yhat = modelo.predict(base_modelo2)
        st.markdown(f"<p class='big-font'>Predicción: {np.round(float(yhat), 2)}</p>", unsafe_allow_html=True)
    except ValueError:
        st.error("Por favor, ingrese valores numéricos válidos en todos los campos.")

# Botón para reiniciar la consulta
if st.button("Reiniciar"):
    st.experimental_rerun()


NameError: name 'st' is not defined